Cat or Dog Neural Network

In [47]:
#load modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import preprocessing
import tensorflow as tf
import os
from PIL import Image
import splitfolders

Image Processing

In [48]:
#goes through each image and checks if it is non-corrupt, deletes if it is corrupted
folder_path = 'PetImages'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        try:
            im = Image.open(file_path)
            rgb_im = im.convert('RGB')
            if filee.split('.')[1] not in extensions:
                extensions.append(filee.split('.')[1])
        except: 
            os.remove(file_path)

Using the above section of code, cats/666.jpg and dogs/11702.jpg were identified as corrupt and deleted

In [49]:
#splits each folder into .8 training, .1 validation, and .1 testing
splitfolders.ratio("PetImages", output="PetImagesSplit",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

In [50]:
"""the following function is used to create a data generator that will be used to train the neural network. The data generator will randomly rotate, shift, and zoom the images to create a more robust neural network."""
training_data_generator = ImageDataGenerator(
        #Rescale the image by 1/255 to normalize the pixel values
        rescale=1.0/255,
        #Randomly increase or decrease the size of the image by up to 20%
        zoom_range=0.2, 
        #Randomly rotate the image between -15,15 degrees
        rotation_range=15, 
        #Shift the image along its width by up to +/- 5%
        width_shift_range=0.05, 
        #Shift the image along its height by up to +/- 5%
        height_shift_range=0.05 
)

In [51]:
#variables being fed to training data generator
train_directory = "PetImagesSplit/train/" #path to the folder containing the images to train
val_directory = "PetImagesSplit/val/" #path to the folder containing the images to val
test_directory = "PetImagesSplit/test/" #path to the folder containing the images to test
class_mode = "categorical"
color_mode = "rgb" 
target_size = (256, 256) #resizes each image to 256x256
batch_size = 16

In [52]:
training_iterator = training_data_generator.flow_from_directory(train_directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)

Found 20731 images belonging to 2 classes.


Mode Building

In [53]:
def create_model_base():
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(256, 256, 3)))
    model.add(tf.keras.layers.Conv2D(2, 5, strides=3, activation="relu")) 
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(5, 5), strides=(5,5)))
    model.add(tf.keras.layers.Conv2D(4, 3, strides=1, activation="relu")) 
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=(2,2), strides=(2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2,activation="softmax"))
    return model

create_model_base was a model used to identify pneumonia in grayscale chest images from a lesson I did. Needless to say, the accuracy was no better than chance after 5 epochs. ~600 parameters.

In [54]:
def create_model_1():
    model = tf.keras.Sequential()   
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 256, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2,activation="softmax"))
    return model

create_model_base1 was based on Jason Brownlee's claim that stacking convolutional layers add easily understandable complexity, which I felt the model definitely needed considering the lower paramter count and accuracy. ~350k params. The accuraccy ended up being ~.55 at 1 and 2 epochs which indicated to me that the model needed even more complexity

In [55]:
def create_model_2():
    model = tf.keras.Sequential()   
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 256, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2,activation="softmax"))
    return model

create_model_complex added an extra layer of convolution and pooling. ~520k params. The added features led to a worse accuraccy of .53-.52 for the first 2 epochs.

In [56]:
def create_model_3():
    model = tf.keras.Sequential()   
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(256, 256, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    # model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    # model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(2,activation="softmax"))
    return model

In [57]:
model = create_model_3()
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 262144)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 524290    
Total params: 543,682
Trainable params: 543,682
Non-trainable params: 0
________________________________________________

In [58]:
#model compilation
#model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])

In [59]:
validation_data_generator = ImageDataGenerator( rescale=1./255)
validation_iterator = validation_data_generator.flow_from_directory(val_directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)
test_data_generator = ImageDataGenerator( rescale=1./255)
test_iterator = validation_data_generator.flow_from_directory(test_directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)

Found 2907 images belonging to 2 classes.
Found 2906 images belonging to 2 classes.


In [60]:
model.fit(
       training_iterator,
       steps_per_epoch=training_iterator.samples/batch_size,
       epochs=2,
       validation_data=validation_iterator,
       validation_steps=validation_iterator.samples/batch_size)

Epoch 1/2
1295/1295 [==============================] - 371s 286ms/step - loss: 0.8841 - accuracy: 0.6488 - val_loss: 0.5558 - val_accuracy: 0.7076
Epoch 2/2
1295/1295 [==============================] - 370s 286ms/step - loss: 0.5707 - accuracy: 0.7077 - val_loss: 0.5041 - val_accuracy: 0.7654


In [61]:
acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
print(acc)

c:\Users\Wiggles\miniconda3\envs\train\lib\site-packages\keras\engine\training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.487257719039917, 0.7670336961746216]
